# Projet électif python: Basket Data

Ce notebook sert de rendu à notre projet d'analyse de données sur la NBA.



---


On commence par importer les librairies utilisés pour l'ensemble du projet:


*   numpy pour la gestion des grands nombres ainsi que certaines fonctions mathématiques.
*   panda pour la récupération des données depuis un fichier `.csv`.
*   plotly pour la représentation graphique des statistiques souhaitées avec un contrôle accru de l'infographie.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

Le code ci-dessous permet de récupérer le fichier .csv source contenant les stats de l'ensemble des joueurs de la NBA (ici la saison 2022-23) et récupère les informations relatives à un joueur précis.

In [2]:
# Chemin vers le fichier CSV
chemin_fichier_csv = "NBA Stats 202324 All Stats  NBA Player Props Tool.csv"
# Importer les données depuis le fichier CSV
donnees = pd.read_csv(chemin_fichier_csv)
nom_joueur = input("Veuillez saisir le nom du joueur : ")
attributs = ['PPG', 'RPG', 'APG', 'BPG', 'SPG', 'TPG', 'TS%']


def recup_stats_joueur(nom_joueur, attributs):
    stats = {}
    # On récupère les lignes correspondant au joueur recherché
    donnees_joueur = donnees[donnees['NAME'] == nom_joueur]
    if not donnees_joueur.empty:
        # Boucler sur chaque attribut
        for attribut in attributs:
            # Filtrer: on retire les valeurs non valides (NaN)
            valeurs_valides = donnees_joueur[attribut].dropna()

            # Vérifier qu'il reste des valeurs valides après le filtrage
            if not valeurs_valides.empty:
                # Calculer la moyenne des valeurs valides et stocker dans le dictionnaire
                moyenne_attribut = valeurs_valides.mean()
                stats[attribut] = moyenne_attribut
            else:
                print(f"Aucune valeur valide trouvée pour la stat: {attribut}.")
    else:
        print("Le joueur", nom_joueur, "n'a pas été trouvé dans les données.")
    return stats

# Appeler la fonction pour récupérer les statistiques du joueur
stats_joueur = recup_stats_joueur(nom_joueur, attributs)

# Afficher les statistiques du joueur
print(f"Stats de {nom_joueur}:")
for attribut, stat in stats_joueur.items():
    print(f"{attribut}: {stat}")

Stats de Rudy Gobert:
PPG: 14.0
RPG: 12.9
APG: 1.3
BPG: 2.1
SPG: 0.7
TPG: 1.6
TS%: 0.675


Cette fonction prend en entrée un `DataFrame` contenant les données des joueurs ainsi qu'une liste d'attributs.
Elle récupère les données pour chaque joueur dans le `DataFrame`, calcule la moyenne de chaque attribut et renvoie un dictionnaire contenant les moyennes de chaque attribut.

In [3]:
def calculer_moyenne_attributs_joueurs(dataframe, attributs):
    moyennes = {}

    # Boucler sur chaque attribut
    for attribut in attributs:
        # Filtrer: on retire les valeurs non valides (NaN)
        valeurs_valides = dataframe[attribut].dropna()

        # Vérifier qu'il reste des valeurs valides après le filtrage
        if len(valeurs_valides) > 0:
            moyenne_attribut = np.mean(valeurs_valides)

            # Ajouter la moyenne de l'attribut au dictionnaire des moyennes
            moyennes[attribut] = moyenne_attribut
        else:
            print(f"Aucune valeur valide trouvée pour la stat: {attribut}.")

    return moyennes

# Calculer la moyenne des attributs pour l'ensemble des joueurs de la liste
moyennes_globales = calculer_moyenne_attributs_joueurs(donnees, attributs)
print("Moyennes globales des stats de l'ensemble des joueurs:")
for attribut, moyenne in moyennes_globales.items():
    print(f"{attribut}: {moyenne}")

Moyennes globales des stats de l'ensemble des joueurs:
PPG: 8.148858447488584
RPG: 3.2777777777777777
APG: 1.9528158295281584
BPG: 0.38843226788432267
SPG: 0.5881278538812785
TPG: 0.9555555555555555
TS%: 0.5372115677321156


Cette fonction reprend le principe de la fonction précédente et calcule les moyennes des statistiques pour les joueurs du même poste que celui du joueur spécifié.

In [4]:
def moyenne_attributs_meme_poste(dataframe, nom_joueur, attributs):
    # Filtrer les données pour n'inclure que les joueurs du même poste que celui spécifié
    poste_joueur = dataframe.loc[dataframe['NAME'] == nom_joueur, 'POS'].iloc[0]
    donnees_meme_poste = dataframe[dataframe['POS'] == poste_joueur]
    # Calcul des moyennes des attributs pour les joueurs
    moyennes_meme_poste = calculer_moyenne_attributs_joueurs(donnees_meme_poste, attributs)
    return moyennes_meme_poste

# Calculer les moyennes des attributs pour les joueurs du même poste que celui du joueur spécifié
moyennes_meme_poste = moyenne_attributs_meme_poste(donnees, nom_joueur, attributs)
# Afficher les moyennes calculées
print("Moyennes des stats pour les joueurs du même poste que", nom_joueur)
for attribut, moyenne in moyennes_meme_poste.items():
    print(f"{attribut}: {moyenne}")

Moyennes des stats pour les joueurs du même poste que Rudy Gobert
PPG: 8.76969696969697
RPG: 5.9787878787878785
APG: 1.5666666666666667
BPG: 0.8772727272727273
SPG: 0.4954545454545455
TPG: 1.0545454545454545
TS%: 0.5689090909090909


La fonction ci-dessous permet de normaliser les statistiques récupérées pour le joueur étudié ainsi que les moyennes calculées. Pour ce faire, on recherche la valeur maximale de chaque stat, tous joueurs confondus, puis on calcul le ratio entre ces deux valeurs pour obtenir un pourcentage.
N.B.: ce traitement permet d'aboutir à des stats en % pour obtenir un diagramme homogène.

In [5]:
def normaliser_donnees(dataframe, joueur, attributs, stats):
    # Créer un dictionnaire pour stocker les valeurs maximales de chaque attribut
    max_valeurs = {attribut: dataframe[attribut].max() for attribut in attributs}
    # Normaliser les données brutes en pourcentage par rapport à la valeur maximale
    donnees_normalisees = {}
    for attribut in attributs:
        # Normaliser les données brutes du joueur
        valeur_joueur = stats[attribut]
        pourcentage_joueur = np.round((valeur_joueur / max_valeurs[attribut]) * 100, 1)  # Arrondir à une décimale
        # Stocker les valeurs normalisées dans le dictionnaire
        donnees_normalisees[attribut] = pourcentage_joueur
    return donnees_normalisees

# Utilisation de la fonction pour normaliser les données pour le joueur spécifié
donnees_normalisees_joueur = normaliser_donnees(donnees, nom_joueur, attributs, stats_joueur)
donnees_normalisees_globales = normaliser_donnees(donnees, nom_joueur, attributs, moyennes_globales)
donnees_normalisees_poste = normaliser_donnees(donnees, nom_joueur, attributs, moyennes_meme_poste)



---


La fonction suivante permet la génération d'un diagramme de Kiviat représentant les statistiques normalisées du joueur séléctionné ainsi que celles des joueurs de toute la ligue, et celles des joueurs du même poste (moyennes). L'objectif est de proposer une comparaison pertinente pour un joueur.

**Remarque :** *Les statistiques affichées sont représentées en % et ne représentent donc pas les valeurs réelles.*

In [6]:
import plotly.graph_objects as go

def diagramme_kiviat(data_joueur, data_moyenne_globale, data_moyenne_poste, joueur):
    """
    Cette fonction crée un diagramme de Kiviat pour visualiser les données normalisées des attributs.
    """
    # Extraire les noms des attributs
    attributs = list(data_joueur.keys())

    # Extraire les valeurs normalisées pour le joueur, la moyenne globale et la moyenne du poste
    valeurs_joueur = list(data_joueur.values())
    valeurs_moyenne_globale = list(data_moyenne_globale.values())
    valeurs_moyenne_poste = list(data_moyenne_poste.values())

    # Créer un diagramme de Kiviat avec Plotly
    fig = go.Figure()

    # Ajouter les traces pour le joueur, la moyenne globale et la moyenne du poste
    fig.add_trace(go.Scatterpolar(
        r=valeurs_joueur,
        theta=attributs,
        fill='toself',
        name=f'{joueur}',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatterpolar(
        r=valeurs_moyenne_globale,
        theta=attributs,
        fill='toself',
        name='Moyenne globale',
        line=dict(color='green')
    ))
    fig.add_trace(go.Scatterpolar(
        r=valeurs_moyenne_poste,
        theta=attributs,
        fill='toself',
        name='Moyenne du poste',
        line=dict(color='orange')
    ))

    # Ajouter un titre et ajuster la mise en page
    fig.update_layout(
        title=f"Comparaison des stats pour {joueur}",
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]  # Plage de l'axe radial en pourcentage
            )
        )
    )

    # Afficher le diagramme
    fig.show()

# Utiliser la fonction pour créer le diagramme de Kiviat
diagramme_kiviat(donnees_normalisees_joueur, donnees_normalisees_globales, donnees_normalisees_poste, nom_joueur)

In [9]:
# Trouver l'équipe du joueur
equipe_joueur = donnees[donnees['NAME'] == nom_joueur]
equipe_joueur = equipe_joueur['TEAM']
# Afficher l'équipe du joueur
print("Équipe de", nom_joueur + ":", equipe_joueur)


chemin_equipe_csv = "NBA Stats 202223 All Stats  NBA Player Props Tool.csv"
# Importer les données depuis le fichier team CSV
team = pd.read_csv(chemin_equipe_csv)
valeurs = ['PPG', 'oEFF', 'dEFF', 'W', 'L', 'pDIFF']


# Récupérer les joueurs par équipe
def meme_equipe(dataframe, nom_joueur):
    # Filtrer les données pour n'inclure que les joueurs de la même équipe que celui spécifié
    equipe_joueur = dataframe.loc[dataframe['NAME'] == nom_joueur, 'TEAM'].iloc[0]
    donnees_meme_equipe = dataframe[dataframe['TEAM'] == equipe_joueur]
    return donnees_meme_equipe
# Afficher les noms des joueurs de la même équipe que celui spécifié
print("Les joueurs de l'équipe de ", nom_joueur, "sont les suivants :")
equipe_joueur = meme_equipe(donnees, nom_joueur)
equipe_joueur = equipe_joueur['NAME']
print(equipe_joueur)


def cinq_majeur(dataframe, nom_joueur):
    # Récupérer les données des joueurs de la même équipe que celui spécifié
    equipe = meme_equipe(dataframe, nom_joueur)
    # Trier les joueurs par ordre décroissant de temps de jeu moyen (MPG)
    equipe_triee = equipe.sort_values(by='MPG', ascending=False)
    # Sélectionner les cinq premiers joueurs avec le temps de jeu moyen le plus élevé
    cinq_majeur = equipe_triee.head(5)
    return cinq_majeur
cinq_joueurs = cinq_majeur(donnees, nom_joueur)
print("Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe sont :")
print(cinq_joueurs[['NAME', 'MPG']])

Équipe de Rudy Gobert: 113    Min
Name: TEAM, dtype: object
Les joueurs de l'équipe de  Rudy Gobert sont les suivants :
12              Anthony Edwards
38           Karl-Anthony Towns
113                 Rudy Gobert
129                    Naz Reid
166                 Mike Conley
186             Jaden McDaniels
253    Nickeil Alexander-Walker
317               Kyle Anderson
378                Monte Morris
396                Shake Milton
428              Troy Brown Jr.
447                  Luka Garza
468                 T.J. Warren
476           Jordan McLaughlin
569                 Daishen Nix
575              Leonard Miller
580                 Josh Minott
629           Wendell Moore Jr.
652              Justin Jackson
Name: NAME, dtype: object
Les cinq joueurs ayant le temps de jeu moyen le plus élevé dans l'équipe sont :
                   NAME   MPG
12      Anthony Edwards  35.1
113         Rudy Gobert  34.1
38   Karl-Anthony Towns  32.7
186     Jaden McDaniels  29.2
166         Mike